# Advanced Usage

In [ ]:
# TODO redo this entire section after we finished the demo.

## Default session

When spannerlog is loaded, a default session (`spannerlog.magic_session`) is created behind the scenes. This is the session that %%spannerlog uses.

Using a session manually enables one to dynamically generate queries, facts, and rules

In [ ]:

#| output: false
import spannerlib
session = spannerlib.magic_session

In [ ]:
#| hide
from nbdev.showdoc import show_doc

In [ ]:
result = session.run_commands('''
    new uncle(str, str)
    uncle("benjen", "jon")
                              ''')

In [ ]:
for maybe_uncle in ['ned', 'robb', 'benjen']:
    result = session.run_commands(f'?uncle("{maybe_uncle}",Y)')

printing results for query 'uncle("ned", Y)':
[]

printing results for query 'uncle("robb", Y)':
[]

printing results for query 'uncle("benjen", Y)':
  Y
-----
 jon



## Changing the session of the magic cells<a class="anchor" id="changing_session"></a>

In cases where you want to work with a custom session, but still make use of the magic system, you can overide the session used by the magic system

In [ ]:
import spannerlib  # default session starts here
from spannerlib import Session

another_session=Session()
old_magic_session = spannerlib.magic_session
spannerlib.magic_session = another_session

In [ ]:
%%spannerlog
# we're now using the new session
new uncle(str, str)
uncle("bob", "greg")
?uncle(X,Y)

printing results for query 'uncle(X, Y)':
  X  |  Y
-----+------
 bob | greg



In [ ]:
# back to the old session
spannerlib.magic_session = old_magic_session
%spannerlog uncle("jim", "dwight")

In [ ]:
print(spannerlib.magic_session._parse_graph)
print(another_session._parse_graph)

(__spannerlog_root) (computed) root
    (0) (computed) relation_declaration: uncle(str, str)
    (1) (computed) add_fact: uncle("benjen", "jon")
    (2) (computed) query: uncle("ned", Y)
    (3) (computed) query: uncle("robb", Y)
    (4) (computed) query: uncle("benjen", Y)
    (5) (computed) add_fact: uncle("jim", "dwight")

(__spannerlog_root) (computed) root
    (0) (computed) relation_declaration: uncle(str, str)
    (1) (computed) add_fact: uncle("bob", "greg")
    (2) (computed) query: uncle(X, Y)



## Mixing magics with dynamic session calls<a class="anchor" id="dynmaic_calls"></a>

Lets take the GPA example from the introductory tutorial.
What if we want to have multiple rules each looking for GPAs of students in different classes.
We wouldnt want to manually write a rule for every single subject.

### python-spannerlog interface functions

we can either write our data manually, or import it from a csv/dataframe:

In [ ]:
%%spannerlog
new lecturer(str, str)
lecturer("rick", "physics")

In [ ]:
from pandas import DataFrame
lecturer_df = DataFrame(([["walter","chemistry"], ["linus", "operating_systems"]]))
session.import_rel(lecturer_df, relation_name="lecturer")

In [ ]:
session.import_rel("sample_data/enrolled.csv", relation_name="enrolled", delimiter=",")

In [ ]:
%%spannerlog
enrolled("abigail", "chemistry")
gpa_str = "abigail 100 jordan 80 gale 79 howard 60"

gpa(Student,Grade) <- py_rgx_string(gpa_str, "(\w+).*?(\d+)")->(Student, Grade),enrolled(Student,X)

?gpa(X,Y)

printing results for query 'gpa(X, Y)':
    X    |   Y
---------+-----
 abigail | 100
 jordan  |  80
  gale   |  79
 howard  |  60



### using spannerlog in python loops

Now we are going to define the rules using a for loop

In [ ]:
subjects = [
    "chemistry",
    "physics",
    "operation_systems",
    "magic",
]

for subject in subjects:
    rule = f"""
    gpa_of_{subject}_students(Student, Grade) <- gpa(Student, Grade), enrolled(Student, "{subject}")
    """
    session.run_commands(rule)
    print(rule)  # we print the rule here to show you what strings are sent to the session


    gpa_of_chemistry_students(Student, Grade) <- gpa(Student, Grade), enrolled(Student, "chemistry")
    

    gpa_of_physics_students(Student, Grade) <- gpa(Student, Grade), enrolled(Student, "physics")
    

    gpa_of_operation_systems_students(Student, Grade) <- gpa(Student, Grade), enrolled(Student, "operation_systems")
    

    gpa_of_magic_students(Student, Grade) <- gpa(Student, Grade), enrolled(Student, "magic")
    


As you can see, we can use the dynamically defined rules in a magic cell

In [ ]:
%%spannerlog
?gpa_of_operation_systems_students(X,Y)

printing results for query 'gpa_of_operation_systems_students(X, Y)':
[]



And we can also query dynamically

In [ ]:
subjects = [
    "chemistry",
    "physics",
    "operation_systems",
    "magic",
]

for subject in subjects:
    query = f"""
    ?gpa_of_{subject}_students(Student, Grade)
    """
    session.run_commands(query)

printing results for query 'gpa_of_chemistry_students(Student, Grade)':
  Student  |   Grade
-----------+---------
  abigail  |     100
  jordan   |      80
  howard   |      60

printing results for query 'gpa_of_physics_students(Student, Grade)':
  Student  |   Grade
-----------+---------
  howard   |      60

printing results for query 'gpa_of_operation_systems_students(Student, Grade)':
[]

printing results for query 'gpa_of_magic_students(Student, Grade)':
[]



## Creating rules Dynamically

here's a more complicated example where we create spannerlog code dynamically:

In [ ]:
from spannerlib import magic_session

%spannerlog new sibling(str, str)
%spannerlog new parent(str, str)
%spannerlog parent("jonathan", "george")
%spannerlog parent("george", "joseph")
%spannerlog parent("joseph", "holy")
%spannerlog parent("holy", "jotaro")
%spannerlog sibling("dio", "jonathan")

a = ["parent", "uncle_aunt", "grandparent", "sibling"]
d = {"uncle_aunt": ["sibling", "parent"], "grandparent": ["parent", "parent"], "great_aunt_uncle": ["sibling", "parent", "parent"]}

for key, steps in d.items():
    # add the start of the rule
    result = key + "(A,Z) <- "
    for num, step in enumerate(steps):
        # for every step in the list, add the condition: step(letter, next letter).
        #  the first letter is always `A`, and the last is always `Z`
        curr_letter = chr(num + ord("A"))
        result += step + "(" + curr_letter + ","
        if (num == len(steps) - 1):
            result += "Z)"
        else:
            result += chr(1 + ord(curr_letter)) + "), "
    print("running:", result)
    magic_session.run_commands(result)
    magic_session.run_commands(f"?{key}(X,Y)")

running: uncle_aunt(A,Z) <- sibling(A,B), parent(B,Z)
printing results for query 'uncle_aunt(X, Y)':
  X  |   Y
-----+--------
 dio | george

running: grandparent(A,Z) <- parent(A,B), parent(B,Z)
printing results for query 'grandparent(X, Y)':
    X     |   Y
----------+--------
 jonathan | joseph
  george  |  holy
  joseph  | jotaro

running: great_aunt_uncle(A,Z) <- sibling(A,B), parent(B,C), parent(C,Z)
printing results for query 'great_aunt_uncle(X, Y)':
  X  |   Y
-----+--------
 dio | joseph



## Processing the result of a query in python and using the result in a new query<a class="anchor" id="query_result_processing"></a>

we can add `format_results=True` to `run_statements` to get the output as one of the following:
1. `[]`, if the result is false,
2. `[tuple()]`, if the result if true (the tuple is empty), or
3. `pandas.DataFrame`, otherwise-

In [ ]:
results = session.run_commands(f'''
    new friends(str, str, str)
    friends("bob", "greg", "clyde")
    friends("steven", "benny", "horace")
    friends("lenny", "homer", "toby")
    ?friends(X,Y,Z)''', print_results=False, format_results=True)

# now we'll showcase processing the result with native python...
# lets filter our tuples with some predicate:
res = results[0].values.tolist()
filtered = tuple(filter(lambda friends: 'bob' in friends or 'lenny' in friends, res))

# and feed the matching tuples into a new query:
session.run_commands('new buddies(str, str)')

for first, second, _ in filtered:
    session.run_commands(f'buddies("{first}", "{second}")')

result = session.run_commands("?buddies(First, Second)")

printing results for query 'buddies(First, Second)':
  First  |  Second
---------+----------
   bob   |   greg
  lenny  |  homer



## Import a relation from a `DataFrame`<a class="anchor" id="import_from_df"></a>

By default, non-boolean query results are saved as a `DataFrame`.
A relation can also be imported from a `DataFrame`, like this:

In [ ]:
from pandas import DataFrame

df = DataFrame([["Shrek",42], ["Fiona", 1337]], columns=["name", "number"])
session.import_rel(df, relation_name="ogres")
%spannerlog ?ogres(X,Y)


printing results for query 'ogres(X, Y)':
   X   |    Y
-------+------
 Shrek |   42
 Fiona | 1337

